In [2]:
import pandas as pd
import seaborn as sns
import numpy as np 

In [5]:
df = pd.read_csv('web-page-phishing.csv')
df.head(10)

,url_length,n_dots,n_hypens,n_underline,n_slash,n_questionmark,n_equal,n_at,n_and,n_exclamation,n_space,n_tilde,n_comma,n_plus,n_asterisk,n_hastag,n_dollar,n_percent,n_redirection,phishing
0,37,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,77,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,126,4,1,2,0,1,3,0,2,0,0,0,0,0,0,0,0,0,1,1
3,18,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,55,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,32,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
6,19,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,81,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
8,42,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,104,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
